In [5]:
import requests
from bs4 import BeautifulSoup
import csv

In [6]:
urls = {"I": "https://www.gutenberg.org/cache/epub/25326/pg25326-images.html", "II": "https://www.gutenberg.org/cache/epub/25759/pg25759-images.html", "III": "https://www.gutenberg.org/cache/epub/26860/pg26860-images.html", "IV": "https://www.gutenberg.org/cache/epub/28420/pg28420-images.html", "V": "https://www.gutenberg.org/cache/epub/28421/pg28421-images.html", "VI": "https://www.gutenberg.org/cache/epub/28422/pg28422-images.html", "VII": "https://www.gutenberg.org/cache/epub/31845/pg31845-images.html", "VIII": "https://www.gutenberg.org/cache/epub/31938/pg31938-images.html", "IX": "https://www.gutenberg.org/cache/epub/32362/pg32362-images.html", "X": "https://www.gutenberg.org/cache/epub/33203/pg33203-images.html"}

In [15]:
# Function to scrape the HTML page and extract page ranges
def scrape_and_extract_page_ranges(url, volume):
    volume_label = "CONTENTS_OF_VOLUME_" + volume
    if volume == 'VI':
        volume_label = "CONTENTS_OF_VOLUME_IV"
    if volume == 'X':
        volume_label = "CONTENTS_OF_VOLUME_IX"
    
    print(volume_label)
    page_ranges = {}
    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <p> tags
    table_elements = soup.find(id=volume_label).findNext('div').find('table').find_all('tr')

    # Iterate over rows
    for row in table_elements:
        # Find all cells in the row
        cells = row.find_all(['td', 'th'])
        # Extract artist name and page number
        if len(cells) >= 2:
            if cells[1].get_text().strip().isdigit():
                artist_name = cells[0].get_text().strip()
                page_number = cells[1].get_text().strip()
                # Find next row to get the end page of the range
                next_row = row.find_next_sibling('tr')
                if next_row:
                    next_cells = next_row.find_all(['td', 'th'])
                    if len(next_cells) >= 2 and next_cells[1].get_text().strip().isdigit():
                        end_page_number = int(next_cells[1].get_text().strip())-1
                        page_ranges[artist_name] = f"{page_number}-{end_page_number}"
            else:
                artist_name = cells[0].get_text().strip()
                page_number = cells[1].get_text().strip()
                page_ranges[artist_name] = f"{page_number}"

    return page_ranges


In [16]:
for volume, url in urls.items():
    pr = scrape_and_extract_page_ranges(url=url, volume=volume)
    
    # Write results to CSV
    with open('biography_page_ranges_' + volume + '.csv', 'w', newline='') as csvfile:
        fieldnames = ['Name', 'Page Range']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
        writer.writeheader()
        for name, page_range in pr.items():
            writer.writerow({'Name': name, 'Page Range': page_range})

CONTENTS_OF_VOLUME_I
CONTENTS_OF_VOLUME_II
CONTENTS_OF_VOLUME_III
CONTENTS_OF_VOLUME_IV
CONTENTS_OF_VOLUME_V
CONTENTS_OF_VOLUME_IV
CONTENTS_OF_VOLUME_VII
CONTENTS_OF_VOLUME_VIII
CONTENTS_OF_VOLUME_IX
CONTENTS_OF_VOLUME_IX
